# <center>Quantum Checkers</center>

### <center>Agha Syed Nasir Mehmood Azeemi, Syed Anus Ali, Salman Muhammad Younus</center>

# Introduction

Quantum Phenomenon has largely remained incomprehensible to the general population, mainly due to the fact that, it is not seen in effect in our day-to-day lives. Quantum Phenomenon can only be observed on a very small scale, the scale of atoms, electrons and all particles alike. However, the strange and unique properties of the quantum world can come in handy for computer scientist and may help them to solve some interesting problems. There are many different kinds problems to solve. Examples include, but are not limited to, searching, sorting, counting, factoring etc. Computer scientists have come to categorize the difficulty of solving these problems in terms of resources (e.g. time, space) required. This is what is known as Computational Complexity Theory. The set of problems that can be solved in polynomial time on a deterministic computer aka classical computer is called the set P. However, there are many differnet problems that computer scientists want to solve but because they lie outside of set P, we can only solve them in exponential time or both exponential time and space on classical computers. Examples include, but are not limited to, Factoring, Traveling Salesperson problem etc. This is what motivates computer scientists to look further from classical computation and devise a new model of computation that can out perform classical compuation. Quantum computation, originally developed to simulate quantum systems, is a way to perform computation by incorporating the strange and unique properties of the qauntum world. Computer scientists have proved that all problems solved by a classical computer can be solved by a quantum computer. However, the question whether quantum computers are indeed faster than classical computer still remains unanswered. There is a glimps of hope though. Quantum algorithms like simons period finding algorithm, Grovers Search algorithm, Shor's algorithm, which all perform better better than their classical counterparts, strentghen our believe that quantum computers are faster than classical computers. This belief is what inspires us to further explore quantum computation. However, to achieve this goal we need as many enthusiatic and brilliant minds as we can find. And what better way to attract people to the field of quantum computing than introducing it's basics in a fun way that sparks curiosity in people.
Hence, the goal of the project is to introduce quantum phenomenon in a fun way to everyone via a game of checkers and maybe some day a brilliant young mind gets interested in the field of quantum computing and ends up proving or dis-proving (hopefully not) that quantum computation is faster than classical compuation.

# Project Description

Strategy games such as chess, Go and Tic-tac-toe have proved to be an excellent medium of introducing quantum phenomenons to the general public in a fun way. These games introduce a quantum twist into these age old games and allow the players to experiment with quantum phenomenons such as superposition, entanglement etc. In this project we aim to bring this fun twist to the game of Checkers.

The aims of the project are listed below:
- Implement and simulate the quantum mechanical effects of entanglement and superposition in the pieces of checker board game.
- Attempt to give an intuitive sense of quantum mechanical effects of entanglement and superposition by:
    - Letting players of the game to play a split move resulting in superposition of states.
    - Letting players interact their pieces in superposition to create entangled states.
- Understand the present available quantum chess implementations to help with building the circuit for checkers.
- Implement the circuit for checkers game with a 6x6 board with 6 pieces for each player.

Some special elements in our Quantum Checkers are as follows:
- Player has a choice to make either a split move (giving superposition states of a piece) or a classical move.
- A piece in superposition or in a classical state can capture the pieces of the other player.
-  When a classical piece $p_0$ tries to capture another piece(s), $p_1$, either in superposition or entangled state, then the classical state of $p_1$ is determined and $p0$ either captures $p_1$ by jumping over it, or occupies $p_1's$ position.
- When a piece in superposition, $p_0$, tries to capture another piece, $p_1$, in superposition then the two pieces are entangled together. The state is only resolved when a third classical piece $p_2$ tries to interact with the entangled pieces.

# Background
The reader does not need to possess an advanced knowledge of quantum computing to understand the contents of this report.
The reader must be familiar with:
- basic quantum concepts such as: 
    - superposition
    - measurement.
- unitary gates which satisfy the property $UU^{\dagger} = I$.
- the dagger operation (transpose of a complex conjugate of a unitary).
- matrix multiplication of a vector with a matrix.

No additional knowledge is required.

In [1388]:
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

## For using custom unitaries as gates
from qiskit import transpile

# Notation and State of the Board:
---

We have chosen the dimensions of 6x6 board which gives us a total of 18 boxes (all black) which can be occupied at any given time as per the checkers rules, each piece can only move diagonally. The boxes have been numbered from bottom right to the top left as shown in the image below.

![board](reference_paper/board_num.jpg)

<br>

The state of the board consists of 36 qubits, meaning each box in the grid is represented in 2 qubits. The first qubit represents the occupancy of the box, 1 in case of being occupied and 0 if not occupied. The second qubit will represent the color of the piece in it, 0 if white and 1 if black.

![board_num](reference_paper/initialstate.jpg)

In our circuit we have chosen to declare a total of 39 qubits with first 18 qubits to carry the occupancy states of each box, next 18 qubits will carry the color states of each corresponding occupancy. The last three qubits are ancillary and are useful for the operations of capture, quantum split and merge. 

Therefore, our state of the board will be represented as:

$$|\psi\rangle = | o_{0}, o_{1}, o_{2}, o_{3} .... o_{17}, c_{0}, c_{1}, c_{2}, c_{3} .... c_{17}, \alpha_{0}, \alpha_{1}, \alpha_{2} \rangle$$
$$\text{where:}$$
$$o_{x} = \text{x'th box's occupancy qubit}$$
$$c_{x} = \text{x'th box's color qubit}$$
$$a_{x} = \text{ancilla qubit}$$


Any move on the board requires atmost 3 and atleast 2 boxes (all diagonal to eachother). For a move involving 2 boxes, the box with piece to move is called the source and is represented as $\ket{s_{o}s_{c}}$, where the subscript defines wether the qubit represents occupancy or color.
<br>
The box where the piece will be moved is called the target and is represented as $\ket{t_{o}t_{c}}$.
<br>
For a move that requires 3 boxes, in addition to source and target box, the third box is referred to as path represented as $\ket{p_{o}p_{c}}$. For this kind of move the path is reffered to the box in between the soruce and the target (diagonally).

In [1389]:
from qiskit import execute, Aer

qcap = QuantumRegister(39, "box")
ccap = ClassicalRegister(36)
bell = QuantumCircuit(qcap, ccap)


### Setting states



## Setting the white pieces occupancy qubits
bell.x(0)
bell.x(1)
bell.x(2)
bell.x(3)
bell.x(4)
bell.x(5)

## Setting the black pieces occupancy qubits
bell.x(12)
bell.x(13)
bell.x(14)
bell.x(15)
bell.x(16)
bell.x(17)


## Setting the black pieces color qubits
bell.x(12+18)
bell.x(13+18)
bell.x(14+18)
bell.x(15+18)
bell.x(16+18)
bell.x(17+18)


## To be used for drawing initial state of board
counts = {'111111000000111111000000000000111111': 20}


bell.draw()

┌───┐
  box_0: ┤ X ├
         ├───┤
  box_1: ┤ X ├
         ├───┤
  box_2: ┤ X ├
         ├───┤
  box_3: ┤ X ├
         ├───┤
  box_4: ┤ X ├
         ├───┤
  box_5: ┤ X ├
         └───┘
  box_6: ─────
              
  box_7: ─────
              
  box_8: ─────
              
  box_9: ─────
              
 box_10: ─────
              
 box_11: ─────
         ┌───┐
 box_12: ┤ X ├
         ├───┤
 box_13: ┤ X ├
         ├───┤
 box_14: ┤ X ├
         ├───┤
 box_15: ┤ X ├
         ├───┤
 box_16: ┤ X ├
         ├───┤
 box_17: ┤ X ├
         └───┘
 box_18: ─────
              
 box_19: ─────
              
 box_20: ─────
              
 box_21: ─────
              
 box_22: ─────
              
 box_23: ─────
              
 box_24: ─────
              
 box_25: ─────
              
 box_26: ─────
              
 box_27: ─────
              
 box_28: ─────
              
 box_29: ─────
         ┌───┐
 box_30: ┤ X ├
         ├───┤
 box_31: ┤ X ├
         ├───┤
 box_32: ┤ X ├
         ├───┤
 box_33: ┤ X ├
         ├───┤
 box_34: ┤ X ├
         ├───┤
 box_35: ┤ X ├
         └───┘
 box_36: ─────
              
 box_37: ─────
              
 box_38: ─────
              
c510: 36/═════

# Movement Design
---

We have designed multiple unitaries to enable the player to make different type of moves in checkers. The classical moves mostly use a simple swap unitary, however the quantum moves use more complex unitaries.

Custom unitaries are used to update the occupancy qubits. The color qubits are updated by using a sequence of common quantum gates such as $CSWAP$, $CNOT$ etc.

When the player makes a move, the relevant occupancy, color and ancilla qubits are input to a circuit that changes their states as per the move.

$$|t_o, s_o, t_c, s_c\rangle = U|t_o, s_o, t_c, s_c\rangle $$




# Gates
---


Classical Checkers has the option of two moves only, the diagonal jump to the nearest diagonal and the capture jump over another piece in the diagonal direction. Our game introduces two new moves in order to demonstrate the effects of superposition and entanglement to the game. These two new moves include the split jump, the puts a piece into superposition of two nearest diagonals and the merge jump that allows a player to return a piece back from super position back into just one box. Hence, we have constructed four unitaries at large in order to implement these four moves:

1. Jump Unitary (Diagonal Jump).
2. Capture (Capture Jump).
3. Uslide (Quantum Split Jump).
4. Umerge (Quantum Merge Jump).

\* Note: This section will introduce the unitaries that will be applied on the occupancy qubits. Therefore, in showing the example move for each unitary, the explanation for the operation on the corresponding color qubits will be skipped. We will explore the full circuit in the next section of the report.

### Custom Unitaries to implement Simple jump, Capture and Quantum Split Jump & Quantum Merge Jump.


- ### Jump Unitary

    The jump unitary implements the classical move of moving 1 diagonal box in checkers. It acts on a space that includes only two boxes, a source and a target. The unitary swaps the source and target qubits, thus imitating a move from one box to another. It is equivalent to the $SWAP$ quantum logic gate.


    ![jump](jump.png)


    $$U_{jump}|t, s\rangle \rightarrow |s, t\rangle$$


In [1390]:
Ujump = [[1, 0, 0, 0],
         [0, 0, 1.j, 0],
         [0, 1.j, 0, 0],
         [0, 0, 0, 1]]



$\newline$
- ### Capture

     The capture unitary acts on a reduced Hilbert space that includes 3 qubits, namely, path, target and source. The path qubit represents the box on the diagonal between the target and the source, since the target and source boxes, in a capture move, are at a distance of 2 boxes in the diagonal. The capture unitary includes checks that would restrict the player from making an illegal move if the conditions for the capture are not met.


     ![split](Capture.png)


     $$U_{capture}|p, t, s\rangle \rightarrow |p', t', s'\rangle$$

     <br>
     There are two cases to consider here; the first in which the path box is occupied and the target box is vacant, meaning the move is valid (right). The second case is in which the path qubit is unoccupied, meaning there is no piece to capture (left).

     ![example_capture](reference_paper/captureex.jpg)

     In the first case, the unitary would give us the correct result, i.e the state after capturing the piece:
     $$U_{capture}|1, 0, 1\rangle \rightarrow |0, 1, 0\rangle$$

     In the second case, the unitary would result in the piece making a simple jump to the next diagonal:
     $$U_{capture}|0, x, 1\rangle \rightarrow |1, x, 0\rangle$$

In [1391]:
Capture=[[1,0,0,0,0,0,0,0],
     [0,0,0,0,1,0,0,0],
     [0,0,0,0,0,1,0,0],
     [0,0,0,0,0,0,1,0],
     [0,1,0,0,0,0,0,0],
     [0,0,1,0,0,0,0,0],
     [0,0,0,1,0,0,0,0],
     [0,0,0,0,0,0,0,1]]


## Checking if the Capture is unitary 
Capturedag=np.matrix(Capture).getH()
print(np.dot(Capture,Capturedag))

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]


## Split jump with path consideration

Before implementing the complete Uslide, we constructed two other unitaries for considerations of a **single path or both paths being blocked**. In case where both paths are open, we use the split jump, when both are blocked the piece stays in position and when only one is open the piece moves in the open direction without going into superposition.

$\newline$
- ### Split

    The split jump unitary allows the player to move their piece into a superposition. It is based on the concept of the Hadamard gate. The resultant state from this unitary would create a superposition, in one of which the piece is present in the left diagonal, and present in the right diagonal in the other superposition state. This is the first gate that introduces the concept of Quantum checkers to the player. The square root in the denominator is used to preserve probabilities. This unitary operates on a 3 qubit space; target 1, target 2, and the source.


    ![jump](split.png)

    $$U_{split}|t_2, t_1, s\rangle \rightarrow |t_2', t_1, s'\rangle + |t_2, t_1', s'\rangle$$

    <br>

    The following case would be successfully achieved by this unitary.

    ![splitex](reference_paper/splitex.png)

    $$U_{split}|0, 0, 1\rangle \rightarrow \frac{i}{\sqrt{2}}\left(|1, 0, 0\rangle + |0, 1, 0\rangle\right)$$

<br>

- ### iSwap

    It is possible that either one of the two desired diagonals for superposition where the player wants to move is blocked. In such a case, the player's piece should only move in one direction where the diagonal is unoccupied. To achieve such a move we would need to consider a path qubit $p_{t}$ in order to execute the following unitary that performs a simple jump.

    ![jump](iswap.png)


  - Case 1, path 1 is open and path 2 is blocked.

  $$U_{iSwap}|p_2,p_1,t_1, t_2, s\rangle \rightarrow |p_2,p_1',t_2, t_1', s'\rangle $$

  - Case 2, path 2 is open and path 1 is blocked.
  
  $$U_{iSwap}|p_2,p_1,t_2, t_1, s\rangle \rightarrow |p_2',p_1,t_2', t_1, s'\rangle $$

    ![splitex](reference_paper/simplejump2.png)

$$U_{split}|0,1,0, 1, 1\rangle \rightarrow |0,1,0, 1, 1\rangle$$

### We apply a Split slide Unitary using the above mentioned gates:

![jump](splitslide.png)

In [1392]:
Uslide = [
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1.j/np.sqrt(2), 1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, -1/np.sqrt(2), 1.j/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1.j/np.sqrt(2), -1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1.j/np.sqrt(2), -1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
]

- ### Merge

    The merge unitary is simply the dagger of the split unitary. It undos the effect of split, that is it collaps the superposition of the piece.

    ![merge](merge.png)

    
    $$U_{merge}\ket{s_{1}, s_{2}, t} \rightarrow \ket{s_{1}', s_{2}', t'}$$

    <br>

    ![merge_example](reference_paper/merge.png)


    $$U_{merge}\ket{1, 1, 0} \rightarrow \ket{0, 0, 1}$$

In [1393]:
Usplit = [[1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1.j, 0, 0, 0],
         [0, 1.j / np.sqrt(2), 1 / np.sqrt(2), 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, -1 / np.sqrt(2), 1.j / np.sqrt(2), 0],
         [0, 1.j / np.sqrt(2), -1 / np.sqrt(2), 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1.j / np.sqrt(2), -1 / np.sqrt(2), 0],
         [0, 0, 0, 1.j, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1]]

Umerge=np.matrix(Usplit).getH()

Used to return state of board as a string

In [1394]:
def print_state_board(counts, size):
    state = []

    counts_temp = {}

    for i in counts:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    # print(counts),

    for i in range(0, 18):

        # i = i//2,

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+18] == '1':
                    if box == " ":
                        box = "B"
                    else:
                        box += "B"
                else:
                    if box == " ":
                        box = "W"
                    else:
                        box += "W"
        
        # print(type(row), type(box))
        row += box + '\t\t'

        count+=1

        if count == 3:
            count = 0
            if count_alt:
                state.append("\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    output = ""
    
    for i in state[:-1]:
        # print(i)
        # print("-----------------" * (size//2))
        output += i + "\n"
        output += ("--------------" * (size//2)) + "\n"
    
    # print(state[-1])
    output += state[-1] + "\n"

    # print(output)
    
    return output

Code to reset the circuit to original starting qubits.

In [1395]:
def reset_circuit():
    global bell, counts
    qr = QuantumRegister(39, "box")
    cr = ClassicalRegister(36)
    bell = QuantumCircuit(qr, cr)

    bell.x(0)
    bell.x(1)
    bell.x(2)
    bell.x(3)
    bell.x(4)
    bell.x(5)

    bell.x(12)
    bell.x(13)
    bell.x(14)
    bell.x(15)
    bell.x(16)
    bell.x(17)


    bell.x(12+18)
    bell.x(13+18)
    bell.x(14+18)
    bell.x(15+18)
    bell.x(16+18)
    bell.x(17+18)

    counts = {'111111000000111111000000000000111111': 20}

Code to create new circuit after measurement using measured bits

In [1396]:
def get_new_circuit(counts):
    qr = QuantumRegister(39, "box")
    cr = ClassicalRegister(36)
    qc = QuantumCircuit(qcap, ccap)
    
    count = 0

    for i in list(counts.keys())[0][0:36]:
        if i ==  '1':
            qc.x(count)
        
        count +=1
    
    return qc

Used to measure the current state of the board

In [1397]:
def measure(s=1):
    global bell
    bell.measure(list(range(36)), list(range(35,-1,-1)))
    # bell.measure([3,4,5,6,7], [4,3,2,1,0])
    job = execute(bell,Aer.get_backend('qasm_simulator'),shots=s)
    global counts
    counts = job.result().get_counts()
    
    # print(counts)
    
    max_count = 0
    state = ""

    for i in counts:
        if counts[i] > max_count:
            max_count = counts[i]
            state = i

    counts = {state: max_count}    

    bell = get_new_circuit(counts)

Updates the state of the board being maintained classically for visualization.

In [1398]:
def update_classical_state(counts, s, t1, t2=None):

    # print(s,t1,t2)

    for i in list(counts.keys())[0:1]:
        temp = list(i)

        temp[t1] = temp[s]
        temp[t1+18] = temp[s+18]

        if t2 != None:
            temp[t2] = temp[s]
            temp[t2+18] = temp[s+18]
        
        temp[s] = '0'
        temp[s+18] = '0'
        
        
        # print(temp)
        
        temp = "".join(temp)
        
        counts[temp] = counts[i]
        del counts[i]
    
    # print_state_board2(counts, 5)
    print("\n\n\n")

# Move Functions
---
Following functions are implemented for each move. The custom unitaries are used to update the occupany bits and some conventional quantum gates are used to update the color qubits of each box according to the action of the unitary.

- ### Jump
    The following circuit implements the classical jump move if it is valid (e.g. the diagnal is un-occupied).
    The swap gates are used to swap the values of source qubits and target qubits.
    The ancilla bit is used as control for the swap gates to make sure the jump is valid. If the occupancy bit of the target is 1, then the swap gates are turned off. The ancilla bit is always return to $\ket{0}$ state.
    
    ![jump_circuit](reference_paper/jump_circuit.jpeg)

In [1399]:
def jump_updated(t1, s, qc):
    # qc.swap(s, t1)
    # qc.swap(s+18, t1+18)

    qc.cx(t1, 36)
    qc.x(36)

    qc.cswap(36, s, t1)
    qc.cswap(36, s+18, t1+18)

    qc.cx(t1, 36)

### Example Move:

Moving the white piece from box 3 to box 7 using a classical jump:

In [1400]:
jump_updated(7, 3, bell)

measure()

print(print_state_board(counts, 6))

	B		B		B
------------------------------------------
B		B		B
------------------------------------------
	 		 		 
------------------------------------------
 		W		 
------------------------------------------
	 		W		W
------------------------------------------
W		W		W



The visual state describes that the white piece has moved diagonally and the board has not entered any superpositional state.

- ### Capture
    The following circuit implements the capture move, which can only be performed classically.
    The capture unitary updates the values of the occupancy bits of source, target and path. We cannot use the same capture unitary to update the color bits because unlike occupancy, the color bits can be $0$ or $1$.
    <br>
    Hence swap gates are used with an ancilla to swap the values of color qubits.
    The ancilla bit is always return to $\ket{0}$ state.

    ![capture_circuit](reference_paper/capture_circuit.jpeg)

In [1401]:
def updateColcapture(t,p,s,ancila, qc):


     qc.swap(s,t)
     qc.cnot(p, ancila)
     qc.cnot(ancila, p)


def capture(t1, s, p, ancila, qc):
     print(f"Capture: t1={t1} s={s} p={p}")
     qc.unitary(Capture, [s, t1, p])
     updateColcapture(t1+18, p+18, s+18, ancila, qc)

### Example Move:

A classical capture by capturing the black piece using the white piece.

In [1402]:
jump_updated(10, 14, bell)

measure()

print(print_state_board(counts, 6))

capture(14, 7, 10, 36, bell)

measure()

print(print_state_board(counts, 6))

	B		B		B
------------------------------------------
B		B		 
------------------------------------------
	 		B		 
------------------------------------------
 		W		 
------------------------------------------
	 		W		W
------------------------------------------
W		W		W

Capture: t1=14 s=7 p=10
	B		B		B
------------------------------------------
B		B		W
------------------------------------------
	 		 		 
------------------------------------------
 		 		 
------------------------------------------
	 		W		W
------------------------------------------
W		W		W



The black piece made use of the classical jump move to move to the consecutive diagonal of the white piece. The white piece, then, executed the capture move. Hence, the black piece's occupancy qubit is changed to 0 and the white piece is moved accordingly. The color qubits have also been updated accordingly.

### Example Move:

A classical capture by capturing the white piece using the black piece.

In [1403]:
jump_updated(7, 4, bell)

jump_updated(9, 7, bell)

measure()

print(print_state_board(counts, 6))

capture(6, 13, 9, 36, bell)

measure()

print(print_state_board(counts, 6))

	B		B		B
------------------------------------------
B		B		W
------------------------------------------
	W		 		 
------------------------------------------
 		 		 
------------------------------------------
	 		 		W
------------------------------------------
W		W		W

Capture: t1=6 s=13 p=9
	B		B		B
------------------------------------------
B		 		W
------------------------------------------
	 		 		 
------------------------------------------
B		 		 
------------------------------------------
	 		 		W
------------------------------------------
W		W		W



This state shows the capture of a white piece by a black piece. The white piece, at box 4, is moved inline diagonally with the black piece at box 13. The black piece executes the capture move and the intended state has been achieved, as seen in the visual output after measurement.

- ### Split

    Ancilla 1 and ancilla 2 qubits are used as path 1 and path 2 qubits. The $CNOT$ gate makes a copy of the target 1 and target occupancy qubits in the ancilla qubits while a copy of source color is saved in ancilla 0. These are then passed as inputs to the Usplit/slide unitary. The path qubits will determine which unitary is to be called as explained in the previous section. If both path qubits are 0, then the split unitary will be called, if either of them is 1 then the iSWAP unitary will be called and the identity unitary will be called when both of them are 1.

    <br>

    After the unitary, the $CSWAP$ gates will move the ancilla qubits into the corresponding color qubits of the targets.

    ![split_circuit](reference_paper/split_slide_circuit.jpeg)

In [1404]:
def updateCColSplit(to1,to2,t1,t2,s,ancila, qc):

     qc.cswap(to1,ancila,t1)
     qc.cswap(to2,ancila,t2)
     



def split_jump_updated(t1,t2,s,ancila, qc):
     # qc.unitary(Usplit,[s, t1, t2])
     qc.cx(t1, 37)
     qc.cx(t2, 38)

     qc.cx(s+18,ancila)
     qc.unitary(Uslide, [s,t1,t2,37,38])
     updateCColSplit(t1, t2, t1+18, t2+18, s, ancila, qc)

### Example Move:

Splitting the black piece at box 15, which has one target blocked, so that it executes a classical jump, and then measuring the state. Then splitting the white piece at box 1.

In [1405]:
measure()

print(print_state_board(counts, 6))

split_jump_updated(12, 13, 15, 36, bell)

measure()

split_jump_updated(3, 4, 1, 36, bell)

update_classical_state(counts, 1, 3, 4)

print(print_state_board(counts, 6))

	B		B		B
------------------------------------------
B		 		W
------------------------------------------
	 		 		 
------------------------------------------
B		 		 
------------------------------------------
	 		 		W
------------------------------------------
W		W		W





	 		B		B
------------------------------------------
B		B		W
------------------------------------------
	 		 		 
------------------------------------------
B		 		 
------------------------------------------
	W		W		W
------------------------------------------
W		 		W



The black piece at box 15 executes a split jump. Since one of the target locations of it is blocked, according to the Uslide unitary defined in the previous section, it will execute a classical jump. The first visual output shows this by measuring the state after the black piece's split jump move.

After that, the white piece at box 1 is split. The state has not been measured after this so the visual output shows the white piece being present at two diagonals, box 3 and 4, which indicates a superposition. An indicator could have been added to pieces that were in superposition to indicate their state, but due to time limitations, that could not be achieved.

- ### Merge

    The following circuit implements the merge move, which is used to collapse two superpositions into one.
    The merge unitary updates the values of the occupancy bits of source, target 1 and target 2, by making the left piece move in the right diagonal and right piece move in the left diagonal, thus the superpositions end up in the same state.
    
    <br>
    
    The color qubits have been swapped using the $CSWAP$ gate. The superpositional state in which the piece is present at source 1 will move the color qubit of it to target and the same will happen with the superpositional state in which the piece is present at source 2.

    ![merge_circuit](reference_paper/merge_circuit.jpeg)

In [1406]:
def merge_updated(s1, s2, t, qc):
    qc.cswap(s1, s1+18, t+18)
    qc.cswap(s2, s2+18, t+18)

    qc.unitary(Umerge, [s1,s2,t])

### Example Move:

Merging the white pieces at boxes 3 and 4 and measuring the state:

In [1407]:
print(print_state_board(counts, 6))

print("\n\n\n")

merge_updated(3, 4, 7, bell)

measure(s=256)

print(print_state_board(counts, 6))

	 		B		B
------------------------------------------
B		B		W
------------------------------------------
	 		 		 
------------------------------------------
B		 		 
------------------------------------------
	W		W		W
------------------------------------------
W		 		W





	 		B		B
------------------------------------------
B		B		W
------------------------------------------
	 		 		 
------------------------------------------
B		W		 
------------------------------------------
	 		 		W
------------------------------------------
W		 		W



The white piece that was split in the previous portion is present is superposition at boxes 3 and 4. The merge move takes those boxes as sources and makes the piece, at the left, move in the right diagonal and vice versa for the piece at the right. Therefore, both superpositional states collapse into one state, as can be seen in the visual output after measurement above. 

Used for finding diagonal boxes in capture function

In [1408]:
up_diag = {0:[-1, 3], 1:[3, 4], 2:[4, 5], 3:[6, 7], 4:[7, 8], 5:[8, -1], 6:[-1, 9], 7:[9, 10], 8:[10, 11], 9:[12, 13], 10:[13, 14], 11:[14, -1], 12:[-1, 15], 13:[15, 16], 14:[16, 17], 15:[-1, -1], 16:[-1, -1], 17:[-1, -1]}

down_diag = {0:[-1, -1], 1:[-1, -1], 2:[-1, -1], 3:[0, 1], 4:[1, 2], 5:[2, -1], 6:[-1, 3], 7:[3, 4], 8:[4, 5], 9:[6, 7], 10:[7, 8], 11:[8, -1], 12:[-1, 9], 13:[9, 10], 14:[10, 11], 15:[12, 13], 16:[13, 14], 17:[14, -1]}

Gets state of board as a string to show in the tkinter UI.

In [1409]:
def print_state_board2(counts, size):
    state = []

    counts_temp = {}

    for i in counts:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    # print(counts),

    for i in range(0, 18):

        # i = i//2,

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+18] == '1':
                    if box == " ":
                        box = "B"
                    else:
                        box += "B"
                else:
                    if box == " ":
                        box = "W"
                    else:
                        box += "W"
        
        # print(type(row), type(box))
        row += box + '\t\t'

        count+=1

        if count == 3:
            count = 0
            if count_alt:
                state.append("\t\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    output = ""
    
    for i in state[:-1]:
        # print(i)
        # print("-----------------" * (size//2))
        output += i + "\n"
        output += ("     " * (size//2)) + ("------------------" * (size//2)) + "\n"
    
    # print(state[-1])
    output += state[-1] + "\n"

    # print(output)
    
    return output

### Tkinter UI

Initializes the tkinter UI for ease in visualization and interaction with the circuit.

In [1410]:
from tkinter import *

root = Tk()  
root.geometry("400x490")  
root.title("test")

# canvas = Canvas(root)

V = StringVar()
label = Label(root, textvariable=V, bg = "white", bd=100, fg = "black")
label.pack()
# label.place(x=-160, y=0)


l1 = Label(root, text="source box: ")
l1.place(x=80, y=360)
s = Entry(root)
s.place(x=150,y=360)
# s.pack()

l2 = Label(root, text="t1 box: ")
l2.place(x=10, y=400)
t1 = Entry(root)
t1.place(x=50, y=400)
# t1.pack()

l3 = Label(root, text="t2 box: ")
l3.place(x=210, y=400)
t2 = Entry(root)
t2.place(x=250, y=400)
# t2.pack()

def gui_jump():
    jump_updated(int(t1.get()), int(s.get()), bell)
    # print(qi.Statevector.from_instruction(bell.reverse_bits()).to_dict())
    update_classical_state(counts, int(s.get()), int(t1.get()))

    V.set(print_state_board2(counts, 6))
    
def gui_sjump():
    split_jump_updated(int(t1.get()), int(t2.get()), int(s.get()), 36, bell)
    # print(qi.Statevector.from_instruction(bell.reverse_bits()).to_dict())
    update_classical_state(counts, int(s.get()), int(t1.get()), int(t2.get()))

    V.set(print_state_board2(counts, 6))

def gui_capture():
    if int(s.get()) < int(t1.get()):
        if (int(t1.get()) - int(s.get())) < 7:
            if up_diag[int(s.get())][0] != -1:
                path_qb = up_diag[int(s.get())][0]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        else:
            if up_diag[int(s.get())][1] != -1:
                path_qb = up_diag[int(s.get())][1]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
                
        # capture(int(t1.get()), int(s.get()), int(s.get())+3, 36, bell)
    else:
        if (int(s.get()) - int(t1.get())) < 7:
            if down_diag[int(s.get())][1] != -1:
                path_qb = down_diag[int(s.get())][1]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        else:
            if down_diag[int(s.get())][0] != -1:
                path_qb = down_diag[int(s.get())][0]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        # capture(int(t1.get()), int(s.get()), int(s.get())-3, 36, bell)

def gui_measure():
    global bell
    bell.measure(list(range(36)), list(range(35,-1,-1)))
    # bell.measure([3,4,5,6,7], [4,3,2,1,0])
    job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1)
    global counts
    counts = job.result().get_counts()
    bell = get_new_circuit(counts)
    print(counts)
    V.set(print_state_board2(counts, 6))

def gui_reset():
    reset_circuit()
    V.set(print_state_board2(counts, 6))

def gui_merge():
    merge_updated(int(s.get()), int(t1.get()), int(t2.get()), bell)

b1 = Button(root, text ="Jump", command=gui_jump)
b1.place(x=120, y=425)

b2 = Button(root, text ="Split Jump", command=gui_sjump)
b2.place(x=170, y=425)
# b.pack()

b3 = Button(root, text ="Capture", command=gui_capture)
b3.place(x=250, y=425)

b5 = Button(root, text ="Merge", command=gui_merge)
b5.place(x=320, y=425)

b4 = Button(root, text ="Measure", command=gui_measure)
b4.place(x=160, y=460)

b4 = Button(root, text ="Reset", command=gui_reset)
b4.place(x=350, y=460)



V.set(print_state_board2(counts, 6))

'''UNCOMMENT THIS TO OPEN USER INTERFACE'''
# root.mainloop()

# Conclusion

To conclude, we were able to implement a game of 6x6 classical checkers with additional quantum moves and rules via a quantum circuit. The game allows a piece to occupy two boxes in superposition, merge two pieces in superposition, capture a piece in superposition and all classical moves with validity checks (e.g. if the move can take place or not). Superposition also allowed us to create entangled qubits where the colour qubits were entangled with the occupancy qubits. Although the circuits work fine for initial moves using the superposition, some erroneous cases have been observed as we play the game. The source of such anomalies seems to be coming from the handling of our colour qubits in relation to occupancy qubits. The GUI of the game is subpar and can be improved a lot. Finally, the design of our implementation is scalable as we are only dealing with six qubits at max or two qubits at least when changing the state of the board. Hence, it is safe to say the aims of the project have been largely met though further refinement is needed.

# Resources

[1] C. Cantwell, “Quantum chess:  Developing a mathematical framework and design methodology forcreating quantum games.”[https://arxiv.org/abs/1906.05836](https://arxiv.org/abs/1906.05836), June 2019.
<br>

[2] “Quantum  chess.”[https://quantumai.google/cirq/experiments/quantum_chess/concepts](https://quantumai.google/cirq/experiments/quantum_chess/concepts), Accessed:  2021-11-5.
<br>

[3]  Kartikeya,    “Quantum-Checkers: A   game   of   checkers   demonstrating   quantum   mechani-cal  phenomenon  (inspired  by  christopher  cantwell’s  quantum  chess).”[https://github.com/VvenomSsnake/Quantum-Checkers](https://github.com/VvenomSsnake/Quantum-Checkers).